# [실습3] IMDB 영화 후기 데이터셋 벡터화

IMDB 데이터셋을 직접 다운로드하여 벡터화하는 과정을 살펴본다.

준비 과정 1: 데이터셋 다운로드 압축 풀기

압축을 풀면 아래 구조의 디렉토리가 생성된다.

```
aclImdb/
...train/
......pos/
......neg/
...test/
......pos/
......neg/
```

`train`의 `pos`와 `neg` 서브디렉토리에 각각 12,500개의 긍정과 부정 후기가
포함되어 있다.

In [ ]:
!curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz

`aclImdb/train/unsup` 서브디렉토리는 필요 없기에 삭제한다.

In [ ]:
import platform

if platform.system() == 'Linux':
    !rm -r aclImdb/train/unsup
else:
    import shutil
    unsup_path = './aclImdb/train/unsup'
    shutil.rmtree(unsup_path)

긍정 후기 하나의 내용을 살펴보자.
모델 구성 이전에 훈련 데이터셋을 살펴 보고
모델에 대한 직관을 갖는 과정이 항상 필요하다.

In [ ]:
if 'google.colab' in str(get_ipython()):
    !cat aclImdb/train/pos/4077_10.txt
else:
    with open('aclImdb/train/pos/4077_10.txt', 'r') as f:
        text = f.read()
        print(text)

준비 과정 2: 검증셋 준비

훈련셋의 20%를 검증셋으로 떼어낸다.
이를 위해 `aclImdb/val` 디렉토리를 생성한 후에
긍정과 부정 훈련셋 모두 무작위로 섞은 후 그중 20%를 검증셋 디렉토리로 옮긴다.

In [ ]:
import os, pathlib, shutil, random

base_dir = pathlib.Path("aclImdb")
val_dir = base_dir / "val"
train_dir = base_dir / "train"

for category in ("neg", "pos"):
    os.makedirs(val_dir / category)            # val 디렉토리 생성
    files = os.listdir(train_dir / category)

    random.Random(1337).shuffle(files)         # 훈련셋 무작위 섞기

    num_val_samples = int(0.2 * len(files))    # 20% 지정 후 검증셋으로 옮기기
    val_files = files[-num_val_samples:]

    for fname in val_files:
        shutil.move(train_dir / category / fname,
                    val_dir / category / fname)

준비 과정 3: 텐서 데이터셋 준비

`text_dataset_from_directory()` 함수를 이용하여
훈련셋, 검증셋, 테스트셋을 준비한다.
자료형은 모두 `Dataset`이며, 배치 크기는 32를 사용한다.

In [ ]:
from tensorflow import keras

batch_size = 32

train_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/train", batch_size=batch_size
    )

val_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/val", batch_size=batch_size
    )

test_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/test", batch_size=batch_size
    )

각 데이터셋은 배치로 구분되며
입력은 `tf.string` 텐서이고, 타깃은 `int32` 텐서이다.
크기는 모두 32이며 지정된 배치 크기이다.
예를 들어, 첫째 배치의 입력과 타깃 데이터의 정보는 다음과 같다.

In [ ]:
for inputs, targets in train_ds:
    print("inputs.shape:", inputs.shape)
    print("inputs.dtype:", inputs.dtype)
    print("targets.shape:", targets.shape)
    print("targets.dtype:", targets.dtype)

    # 예제: 첫째 배치의 첫째 후기
    print("inputs[0]:", inputs[0])
    print("targets[0]:", targets[0])

    break

### 11.3.3 시퀀스 활용법

**정수 벡터 데이터셋 준비**

훈련셋의 모든 후기 문장을 정수들의 벡터로 변환한다.
단, 후기 문장이 최대 600개의 단어만 포함하도록 한다.
또한 사용되는 어휘는 빈도 기준 최대 2만 개로 제한한다.

- `max_length = 600`
- `max_tokens = 20000`
- `output_sequence_length=max_length`

In [ ]:
from tensorflow.keras import layers

max_length = 600
max_tokens = 20000

text_vectorization = layers.TextVectorization(
    max_tokens=max_tokens,
    output_mode="int",
    output_sequence_length=max_length,
)
# 어휘 색인 생성 대상 훈련셋 후기 텍스트 데이터셋
text_only_train_ds = train_ds.map(lambda x, y: x)

text_vectorization.adapt(text_only_train_ds)

int_train_ds = train_ds.map(lambda x, y: (text_vectorization(x), y))
int_val_ds = val_ds.map(lambda x, y: (text_vectorization(x), y))
int_test_ds = test_ds.map(lambda x, y: (text_vectorization(x), y))

변환된 첫째 배치의 입력과 타깃 데이터의 정보는 다음과 같다.
`output_sequence_length=600`으로 지정하였기에 모든 문장은 단어를 최대 600개에서
잘린다. 따라서 생성되는 정수들의 벡터는 길이가 모두 600으로 지정된다.
물론 문장이 600개보다 적은 수의 단어를 사용한다면 나머지는 0으로 채워진다.
또한 벡터에 사용된 정수는 2만보다 작은 값이며,
이는 빈도가 가장 높은 2만개의 단어만을 대상(`max_tokens=20000`)으로 했기 때문이다.

In [ ]:
for inputs, targets in int_train_ds:
    print("inputs.shape:", inputs.shape)
    print("inputs.dtype:", inputs.dtype)
    print("targets.shape:", targets.shape)
    print("targets.dtype:", targets.dtype)
    print("inputs[0]:", inputs[0])
    print("targets[0]:", targets[0])
    break

**트랜스포머 구현**

위 그림에서 설명된 트랜스포머 인코더를 층(layer)으로 구현하면 다음과 같다.

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"),
             layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()

    def call(self, inputs, mask=None):
        if mask is not None:
            mask = mask[:, tf.newaxis, :]
        attention_output = self.attention(
            inputs, inputs, attention_mask=mask)
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)

    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config

**트랜스포머 인코더 활용 모델**

훈련 데이터셋이 입력되면 먼저 단어 임베딩을 이용하여
단어들 사이의 연관성을 찾는다.
이후 트랜스포머 인코더로 셀프 어텐션을 적용한다.

In [ ]:
vocab_size = 20000
embed_dim = 256
num_heads = 2
dense_dim = 32

inputs = keras.Input(shape=(None,), dtype="int64")

x = layers.Embedding(vocab_size, embed_dim)(inputs)
x = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)

# 길이가 600인 1차원 어레이로 변환
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dropout(0.5)(x)

outputs = layers.Dense(1, activation="sigmoid")(x)

model = keras.Model(inputs, outputs)

model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])

model.summary()

훈련 과정은 특별한 게 없다.
테스트셋에 대한 정확도가 87.5% 정도로 바이그램 모델보다 좀 더 낮다.

In [ ]:
callbacks = [
    keras.callbacks.ModelCheckpoint("transformer_encoder",
                                    save_best_only=True)
]

model.fit(int_train_ds, validation_data=int_val_ds, epochs=20, callbacks=callbacks)

model = keras.models.load_model(
    "transformer_encoder",
    custom_objects={"TransformerEncoder": TransformerEncoder})

print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

**단어 위치 인코딩**

다음 `PositionalEmbedding` 층 클래스는 두 개의 임베딩 클래스를 사용한다.
하나는 보통의 단어 임베딩이며,
다른 하나는 단어의 위치 정보를 임베딩한다.
각 임베딩의 출력값을 합친 값을 트랜스포머에게 전달하는 역할을 수행한다.

In [ ]:
class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, input_dim, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.token_embeddings = layers.Embedding(
            input_dim=input_dim, output_dim=output_dim)
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=output_dim)
        self.sequence_length = sequence_length
        self.input_dim = input_dim
        self.output_dim = output_dim

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)

    def get_config(self):
        config = super().get_config()
        config.update({
            "output_dim": self.output_dim,
            "sequence_length": self.sequence_length,
            "input_dim": self.input_dim,
        })
        return config

**단어위치인식 트랜스포머 아키텍처**

아래 코드는 `PositionalEmbedding` 층을 활용하여 트랜스포머 인코더가
단어위치를 활용할 수 있도록 한다.

In [ ]:
vocab_size = 20000
sequence_length = 600
embed_dim = 256
num_heads = 2
dense_dim = 32

inputs = keras.Input(shape=(None,), dtype="int64")

x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(inputs)
x = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)

x = layers.GlobalMaxPooling1D()(x)
x = layers.Dropout(0.5)(x)

outputs = layers.Dense(1, activation="sigmoid")(x)

model = keras.Model(inputs, outputs)

In [ ]:
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])

model.summary()

In [ ]:
callbacks = [
    keras.callbacks.ModelCheckpoint("full_transformer_encoder",
                                    save_best_only=True)
]

model.fit(int_train_ds, validation_data=int_val_ds, epochs=20, callbacks=callbacks)

model = keras.models.load_model(
    "full_transformer_encoder",
    custom_objects={"TransformerEncoder": TransformerEncoder,
                    "PositionalEmbedding": PositionalEmbedding})
print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")